In [ ]:
import traitlets as T

import ipywidgets as W
from ipyradiant import MultiPanelSelect, PathLoader
from rdflib import Graph

## Start with creating a class composing of a checkbox, and the multipanel select.

In [ ]:
class objectLiteralApp(W.VBox):
    graph = T.Instance(Graph, allow_none=True)
    literal_checkbox = T.Instance(W.Checkbox)
    multiselect = T.Instance(MultiPanelSelect)
    check_value = T.Bool()

    def __init__(self, graph: Graph = None, *args, **kwargs):
        if graph is not None:
            kwargs["graph"] = graph
        if "layout" not in kwargs:
            kwargs["layout"] = {
                "min_height": "400px",
                "flex": "1",
            }
        super().__init__(*args, **kwargs)

        self.children = [self.multiselect, self.literal_checkbox]

    @T.default("graph")
    def make_default_graph(self):
        return Graph()

    @T.default("literal_checkbox")
    def make_default_checkbox(self):
        style = {"description_width": "initial"}
        checkbox = W.Checkbox(
            description="Add predicates where object is literal", style=style
        )
        T.link((checkbox, "value"), (self, "check_value"))
        return checkbox

    @T.default("multiselect")
    def make_default_multiselect(self):
        return MultiPanelSelect(data=[])

    @T.observe("check_value")
    def populate_graph(self, change):
        if self.check_value == True:
            self.multiselect.data = [1, 2, 3]
            query1 = """
                SELECT DISTINCT  ?p 
                WHERE {
                        ?s ?p ?o .
                    FILTER (!isLiteral(?o))
                    } 
                """
            q1 = self.graph.query(query1)
            results = []
            for result in q1:
                results.append(result[0])

            query2 = """
                SELECT DISTINCT  ?p 
                WHERE {
                        ?s ?p ?o .
                    FILTER isLiteral(?o)
                    } 
                """
            q2 = self.graph.query(query2)
            final_results = []
            for result in q2:
                if result[0] not in results:
                    final_results.append(result[0])
            self.multiselect = MultiPanelSelect(data=final_results)
            self.children = [self.multiselect, self.literal_checkbox]

We can borrow the loading widgets seen in other examples like the Tab App in order to
get a graph to play around with.

In [ ]:
from ipyradiant import FileManager

lw = FileManager(loader=PathLoader(path="data"))
lw

In [ ]:
graph = lw.graph
ola = objectLiteralApp(graph=graph)
ola